In [69]:
import json

# 假设这两个文件已经存在，并且它们的路径是已知的
file_path1 = 'extracted_cosql_info.json'
file_path2 = 'cosql增强数据.json'

# 读取两个JSON文件
with open(file_path1, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)

with open(file_path2, 'r', encoding='utf-8') as file2:
    data2 = json.load(file2)

# 准备一个新的列表来存储整合后的数据
integrated_data = []

# 定义一个函数来处理turns字段
def process_turns(turns):
    processed_turns = []
    for turn in turns:
        # 只保留isUser字段和text或query字段
        processed_turn = {'type': turn.get('type',''),'isUser': turn.get('isUser','')}
        query = turn.get('query','').replace('\n',' ')
        if query!='':
            processed_turn['query'] = query
        else:
            processed_turn['text'] = turn['text']
        processed_turns.append(processed_turn)

    return processed_turns

# 将两个数据集合并在一起
for item in data1 + data2:
    # 提取db_name字段
    db_name = item['db_name']
    # 处理turns字段
    turns = process_turns(item['turns'])

    # 删除query字段中不以select开头的元素及其上一个元素
    i = 0
    while i < len(turns):
        if 'query' in turns[i] and not turns[i]['query'].lower().startswith('select'):
            if i > 0:
                del turns[i-1]
                i -= 1
            del turns[i]
        else:
            i += 1
    for i, turn in enumerate(turns):
        if 'type' in turn and turn['type'] == 'AMBIGUOUS':
            if(i+1<len(turns)):
                if turns[i+1].get('type','') != 'CLARIFY':
                    del turns[i:i+3]
                    break
            if(i+3<len(turns)):
                if turns[i+3].get('type','') != 'CONFIRM_SQL':
                    del turns[i:i+3]
                    break
                if i+2 < len(turns):
                    combined_text = ' | '.join([turns[i+0].get('text',''),turns[i+1].get('text',''),turns[i+2].get('text','')] )
                    # 删除被合并的元素
                    del turns[i:i+3]
                    # 将合并后的text添加到turns中
                    turns.insert(i, {'type':turn.get('type',''),'isUser': True, 'text': combined_text})
                
    # 删除除了INFORM_SQL、INFER_SQL、AMBIGUOUS和CONFIRM_SQL之外的所有元素
    turns = [turn for turn in turns if turn['type'] in ['INFORM_SQL', 'INFER_SQL', 'AMBIGUOUS', 'CONFIRM_SQL']]

    # 检查是否有AMBIGUOUS type字段，并且如果AMBIGUOUS之后的元素不是CONFIRM_SQL，则删除这个AMBIGUOUS的元素
    i = 0
    while i < len(turns):
        if turns[i]['type'] == 'AMBIGUOUS':
            if i+1 < len(turns) and turns[i+1]['type'] != 'CONFIRM_SQL':
                del turns[i]
                continue
            if i+1 == len(turns):
                del turns[i]
                continue
        i += 1

    # 对final_turns进行修改 idx为单数的如果type字段不是CONFIRM_SQL则删除它和它前一个元素
    final_turns = []
    for i, turn in enumerate(turns):
        # idx为1 3 5...的
        if i % 2 != 0:
            # 如果type字段不是CONFIRM_SQL
            if turn['type'] != 'CONFIRM_SQL' or turn.get('query','')=='':
                # 删除它和它前一个元素
                if i > 0:
                    del turns[i-1:i+1]
                continue
        elif turn['type'] == 'CONFIRM_SQL' and not turn.get('query','').lower().startswith('select'):
            # 删除它和它前一个元素
            if i > 0:
                del turns[i-1:i+1]
                continue
        final_turns.append(turn)

    if len(final_turns) == 0:
        continue

    # 将处理后的db_name和turns添加到新的列表中
    integrated_data.append({'db_name': db_name, 'turns': final_turns})

# 将整合后的数据转换为JSON格式
integrated_json = json.dumps(integrated_data, ensure_ascii=False, indent=4)

# 保存到新的JSON文件中
output_file_path = 'multi_eval/SQLtask数据集.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(integrated_json)

print(output_file_path)


multi_eval/SQLtask数据集.json
